In [2]:
import pandas as pd
import cv2
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

base_dir = './open/test/'

In [3]:
csv_dir = './results/fold1234_ft_nms.csv'
df = pd.read_csv(csv_dir)
df

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,064442001.png,16,0.97468,1138,181,1429,181,1429,496,1138,496
1,064507368.png,16,0.85556,492,124,761,124,761,429,492,429
2,065131036.png,3,0.93839,1108,48,1359,48,1359,314,1108,314
3,065131036.png,25,0.57963,837,440,1171,440,1171,853,837,853
4,065147868.png,16,0.96906,727,188,1010,188,1010,500,727,500
...,...,...,...,...,...,...,...,...,...,...,...
3704,183744509.png,1,0.97477,910,697,1204,697,1204,984,910,984
3705,183911746.png,7,0.97298,508,800,826,800,826,1079,508,1079
3706,183911746.png,27,0.96981,751,280,1003,280,1003,562,751,562
3707,183928410.png,27,0.96797,684,47,919,47,919,308,684,308


In [4]:
def csv2dict(df):
    image_dict = defaultdict(list)
    for i in range(len(df)):
        file_name, class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = df.iloc[i]
        image_dict[file_name].append([class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y])
    
    for k,v in image_dict.items():
        image_dict[k] = sorted(v, key=lambda x: x[1], reverse=True)

    return image_dict

image_dict = csv2dict(df)
len(image_dict.keys())


3251

In [5]:
df = defaultdict(list)
for image_name, infos in tqdm(image_dict.items()):
    for info in infos:
        class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = info
        df['crop_name'].append(image_name)
        df['class_id'].append(class_id)
        df['confidence'].append(confidence)
        df['point1_x'].append(point1_x)
        df['point1_y'].append(point1_y)
        df['point2_x'].append(point2_x)
        df['point2_y'].append(point2_y)
        df['point3_x'].append(point3_x)
        df['point3_y'].append(point3_y)
        df['point4_x'].append(point4_x)
        df['point4_y'].append(point4_y)

df = pd.DataFrame(df)
df


  0%|          | 0/3251 [00:00<?, ?it/s]

100%|██████████| 3251/3251 [00:00<00:00, 854151.99it/s]


,crop_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,064442001.png,16,0.97468,1138,181,1429,181,1429,496,1138,496
1,064507368.png,16,0.85556,492,124,761,124,761,429,492,429
2,065131036.png,3,0.93839,1108,48,1359,48,1359,314,1108,314
3,065131036.png,25,0.57963,837,440,1171,440,1171,853,837,853
4,065147868.png,16,0.96906,727,188,1010,188,1010,500,727,500
...,...,...,...,...,...,...,...,...,...,...,...
3704,183744509.png,1,0.97477,910,697,1204,697,1204,984,910,984
3705,183911746.png,7,0.97298,508,800,826,800,826,1079,508,1079
3706,183911746.png,27,0.96981,751,280,1003,280,1003,562,751,562
3707,183928410.png,27,0.96797,684,47,919,47,919,308,684,308


In [6]:
# test_df 만들기
croppedimages = defaultdict(list) # 원본 이미지명 : [[잘린 이미지 명, 이미지 파일, 클래스], ...]

for image_name, infos in tqdm(image_dict.items(), total=len(image_dict.keys())):
    image_path = base_dir + image_name
    # image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    for i, info in enumerate(infos):
        class_id, confidence, point1_x, point1_y, point2_x, point2_y, point3_x, point3_y, point4_x, point4_y = info
        xmin, ymin, xmax, ymax = point1_x, point1_y, point3_x, point3_y

        xmin, ymin, xmax, ymax = point1_x, point1_y, point3_x, point3_y
        croppedimages[image_name].append([f'{image_name.replace(".png","")}_{i}_{class_id}_{xmin}_{ymin}_{xmax}_{ymax}.png', class_id, confidence])
        # croppedimages[image_name].append([f'{image_name.replace(".png","")}_{i}_{class_id}_{xmin}_{ymin}_{xmax}_{ymax}.png', image[int(point1_y):int(point3_y), int(point1_x):int(point3_x)], class_id])
 

100%|██████████| 3251/3251 [00:00<00:00, 388127.13it/s]


In [11]:
# cropped 이미지 저장할 폴더 생성
csv_name = csv_dir.split('/')[-1].split('.')[0]
save_dir = f'./{csv_name}_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    

In [12]:
# test crop 이미지 저장
for image_name, infos in tqdm(croppedimages.items(), total=len(croppedimages.keys())):
    for info in infos:
        name, image, label = info
        image_save_dir = os.path.join(save_dir, name)
        cv2.imwrite(image_save_dir, image)

100%|██████████| 3251/3251 [00:28<00:00, 115.57it/s]
